# Limpieza y Modelado de Datos

Este notebook implementa:
- Validación de tipos de datos
- Validación de rangos y dominios
- Deduplicación (clave natural + último gana por _ingest_ts)
- Cálculos de negocio (importe total)

In [ ]:
import pandas as pd
import sqlite3
from pathlib import Path
import pyarrow.parquet as pq

# Configuración de rutas
PROJECT_ROOT = Path().resolve().parent
OUTPUT_DIR = PROJECT_ROOT / "output"
PARQUET_DIR = OUTPUT_DIR / "parquet"

# Conexión a SQLite
DB_PATH = OUTPUT_DIR / "ventas.db"
conn = sqlite3.connect(DB_PATH)

In [ ]:
# Leer datos de Parquet
ventas_df = pd.concat([
    pd.read_parquet(f) 
    for f in PARQUET_DIR.glob('ventas/**/*.parquet')
])

# Convertir tipos de datos
ventas_df['fecha'] = pd.to_datetime(ventas_df['fecha'])
ventas_df['unidades'] = pd.to_numeric(ventas_df['unidades'])
ventas_df['precio_unitario'] = pd.to_numeric(ventas_df['precio_unitario'])
ventas_df['_ingest_ts'] = pd.to_datetime(ventas_df['_ingest_ts'])

# Eliminar duplicados (último gana por _ingest_ts)
ventas_df = (
    ventas_df
    .sort_values('_ingest_ts', ascending=True)  # Ordenar por timestamp
    .drop_duplicates(
        subset=['fecha', 'id_cliente', 'id_producto'],  # Clave natural
        keep='last'  # Mantener última versión
    )
)

# Calcular importe total
ventas_df['importe_total'] = ventas_df['unidades'] * ventas_df['precio_unitario']

# Guardar datos limpios en SQLite
ventas_df.to_sql('clean_ventas', conn, if_exists='replace', index=False)

# Guardar datos limpios en Parquet
table = pa.Table.from_pandas(ventas_df)
pq.write_to_dataset(
    table,
    root_path=str(PARQUET_DIR / 'ventas_clean'),
    partition_cols=['year', 'month']
)

print("Dimensiones del dataset limpio:", ventas_df.shape)
print("\nEstadísticas descriptivas:")
print(ventas_df.describe())